In [ ]:
import cv2
from cv2 import aruco
import numpy as np

import os
import yaml
import pickle
import glob

import time

from google.colab import drive
drive.mount('/content/drive')

from google.colab.patches import cv2_imshow

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Code developed by abakisita and t-walker-21. *Edited by Thomas Yingling*

In [ ]:
directory_board = r"/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/bin/boards"

##### Intrinsic
# Use a predefined dictionary
# and for a bunch of fiducial markers
intrinsic_dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_250)

# Generate a Chessboard pattern + ArUco pattern board
# Input params:
# Num. squares in X direction = 6
# Num. squares in Y direction = 8
# Chess board square length = 1 inch
# Aruco marker square length = 1/2 inch
# NOTE: Make sure the prints align to this "real-world" dimensions
intrinsic_num_x = 6
intrinsic_num_y = 8
intrinsic_checkerboard_width = 1.274 * 25.4 / 1000 # inch -> mm -> m
intrinsic_marker_width = intrinsic_checkerboard_width / 2
intrinsic_board = cv2.aruco.CharucoBoard_create(
    intrinsic_num_x, intrinsic_num_y, intrinsic_checkerboard_width, intrinsic_marker_width, intrinsic_dictionary)

# Number parameters
INTRINSIC_MAX_ARUCO_IDS = intrinsic_num_x * intrinsic_num_y // 2
INTRINSIC_MAX_CHARUCO_IDS = (intrinsic_num_x - 1) * (intrinsic_num_y - 1)

##### Extrinsic
# Use a predefined dictionary
# and for a bunch of fiducial markers
extrinsic_dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_250)

# Generate a Chessboard pattern + ArUco pattern board
# Input params:
# Num. squares in X direction = 4
# Num. squares in Y direction = 6
# Chess board square length = 9 inch
# Aruco marker square length = 9/2 inch
# NOTE: Make sure the prints align to this "real-world" dimensions
extrinsic_num_x = 6
extrinsic_num_y = 8
extrinsic_checkerboard_width = 1.131 * 25.4 / 1000 # inch -> mm -> m
extrinsic_marker_width = extrinsic_checkerboard_width / 2
extrinsic_board = cv2.aruco.CharucoBoard_create(
    extrinsic_num_x, extrinsic_num_y, extrinsic_checkerboard_width, extrinsic_marker_width, extrinsic_dictionary)

# Number parameters
EXTRINSIC_MAX_ARUCO_IDS = extrinsic_num_x * extrinsic_num_y // 2
EXTRINSIC_MAX_CHARUCO_IDS = (extrinsic_num_x - 1) * (extrinsic_num_y - 1)


def main():
    # Draw the intrinsic board on a canvas of
    # 600 x 800 px with some borders
    # NOTE: on a 8.5 x 11" sheet of paper:
    # Squares: 2.65cm^2
    # Markers: 1.3cm^2 w/ 0.3cm^2 openings
    # Board: 15.8cm x 21.2cm ; 6 x 8 squares
    intrinsic_board_img = intrinsic_board.draw((600, 800), 10, 1)

    # Save the image
    # cv2.imwrite('calib_intrinsic_board.png', intrinsic_board_img)
    cv2.imwrite(os.path.join(directory_board , 'calib_intrinsic_board.png'), intrinsic_board_img)

    # Draw the extrinsic board on a canvas of
    # 3000 x 5000 px with some borders
    extrinsic_board_img = extrinsic_board.draw((4000, 6000), 100, 1)

    # Save the image
    # cv2.imwrite('calib_extrinsic_board.png', extrinsic_board_img)
    cv2.imwrite(os.path.join(directory_board , 'calib_extrinsic_board.png'), extrinsic_board_img)

if __name__ == '__main__':
    main()

## Code developed by kyle-bersani. *Edited by Thomas Yingling*
---



In [ ]:
# ChAruco dictionary and board pull
ARUCO_DICT = intrinsic_dictionary
CHARUCO_BOARD = intrinsic_board
ARUCO_PARAMS = aruco.DetectorParameters_create()

# Create the arrays and variables we'll use to store info like corners and IDs from images processed
corners_all = [] # Corners discovered in all images processed
ids_all = [] # Aruco ids corresponding to corners discovered
image_size = None # Determined at runtime
counter = 0 # Counter for naming convention and loop iterations

directory_test_images = r"/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/test_images/*"
directory_imgs_calibrated = r"/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/bin/imgs_w_cali"
directory_pickle_yam = r"/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/bin/pickle_yam_files"

# This requires a set of images or a video taken with the camera you want to calibrate
# I'm using a set of images taken with the camera with the naming convention:
# 'IMG_<NUMBER>.jpg'
# All images used should be the same size, which if taken with the same camera shouldn't be a problem
images = glob.glob(directory_test_images)

# Loop through images glob'ed
for iname in images:
    # Increment Counter
    counter = counter + 1
    print(counter)

    # Open the image
    img = cv2.imread(iname)

    # Grayscale the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find aruco markers in the query image
    corners, ids, _ = aruco.detectMarkers(
            image=gray,
            dictionary=ARUCO_DICT,
            parameters=ARUCO_PARAMS)

    # Outline the aruco markers found in our query image
    img = aruco.drawDetectedMarkers(
            image=img, 
            corners=corners)

    # Get charuco corners and ids from detected aruco markers
    response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD)

    # If a Charuco board was found, let's collect image/corner points
    # Requiring at least 20 squares
    if response > 20:
        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)
        
        # Draw the Charuco board we've detected to show our calibrator the board was properly detected
        img = aruco.drawDetectedCornersCharuco(
                image=img,
                charucoCorners=charuco_corners,
                charucoIds=charuco_ids)
       
        # Uncomment If the image size is unknown, set it now
        if not image_size:
            image_size = gray.shape[::-1]
    
        # Uncomment to reproportion the image, maxing width or height at a number (like a 1000)
        proportion = max(img.shape) / 1000
        img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))

        # Save the image
        # cv2.imwrite('charuco_img.png', img)
        cv2.imwrite(os.path.join(directory_imgs_calibrated , 'charuco_img_' +str(counter)+ '.jpg'), img)

        # Pause to display each image, waiting for key press
        cv2_imshow(img)
        cv2.waitKey(0)
    else:
        print("Not able to detect a charuco board in image: {}".format(iname))

# Destroy any open CV windows
cv2.destroyAllWindows()

# Make sure at least one image was found
if len(images) < 1:
    # Calibration failed because there were no images, warn the user
    print("Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file.")
    # Exit for failure
    exit()

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
    # Exit for failure
    exit()

# Now that we've seen all of our images, perform the camera calibration
# based on the set of points we've discovered
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)
    
# Print matrix and distortion coefficient to the console
print(cameraMatrix)
print(distCoeffs)
    
# Save values to be used where matrix+dist is required, for instance for posture estimation
# Saved files into a pickle file. However, you can use yaml or whatever works for you
f = open(directory_pickle_yam + '/calibration.pckl', 'wb')
pickle.dump((cameraMatrix, distCoeffs, rvecs, tvecs), f)
f.close()
    
# Print to console our success
print('Calibration successful. Calibration file used: {}'.format('calibration.pckl'))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# The following code is used to watch a video stream, detect a Charuco board, and use
# it to determine the posture of the camera in relation to the plane
# of markers.
#
# Assumes that all markers are on the same plane, for example on the same piece of paper
#
# Requires camera calibration

counter = 0 # For file naming convention, loop iterations, and loop stop condition
image_size = None # Determined at runtime
img_array = [] # To string together images and create a video

# List of Directories to store files
directory_pose = r"/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/bin/vid_frames_w_pose"
directory_vid_in = "/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/test_video/aruco_vid_.mp4"
directory_vid_out = "/content/drive/MyDrive/Colab Notebooks/calibration_testing_v1/bin/pose_vid_out/"

# Check for camera calibration data
if not os.path.exists(directory_pickle_yam + '/calibration.pckl'):
    print("You need to calibrate the camera you'll be using. See calibration project directory for details.")
    exit()
else:
    f = open(directory_pickle_yam + '/calibration.pckl', 'rb')
    (cameraMatrix, distCoeffs, _, _) = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Calibration issue. Remove ./calibration.pckl and recalibrate your camera with CalibrateCamera.py.")
        exit()

# Create vectors we'll be using for rotations and translations for postures
rvecs, tvecs = None, None

# Video Info
cam = cv2.VideoCapture(directory_vid_in)  # Capture input video
frame_rate = 10   # Set Frames video should downsample to
prev = 0          # Timing Parameter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # File Format of output video

while(cam.isOpened()):
    
    # stagger the frames of the video with time.time()
    time_elapsed = time.time() - prev

    if time_elapsed > 1./frame_rate:
        prev = time.time()

        counter=counter+1
        print(counter)

        # Capturing each frame of our video stream
        # ret sets to false at end of video
        ret, QueryImg = cam.read()

        if ret == False:
            break
        if ret == True:
            # grayscale image
            gray = cv2.cvtColor(QueryImg, cv2.COLOR_BGR2GRAY)
        
            # Detect Aruco markers
            corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, ARUCO_DICT, parameters=ARUCO_PARAMS)
      
            # Refine detected markers
            # Eliminates markers not part of our board, adds missing markers to the board
            corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
                    image = gray,
                    board = CHARUCO_BOARD,
                    detectedCorners = corners,
                    detectedIds = ids,
                    rejectedCorners = rejectedImgPoints,
                    cameraMatrix = cameraMatrix,
                    distCoeffs = distCoeffs)   

            # Outline all of the markers detected in our image
            QueryImg = aruco.drawDetectedMarkers(QueryImg, corners, borderColor=(0, 0, 255))

            # Only try to find CharucoBoard if we found markers
            if ids is not None and len(ids) > 10:

                # Get charuco corners and ids from detected aruco markers
                response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
                        markerCorners=corners,
                        markerIds=ids,
                        image=gray,
                        board=CHARUCO_BOARD)
        
                # Require more than 20 squares
                if response is not None and response > 20:
                    # Estimate the posture of the charuco board, which is a construction of 3D space based on the 2D video 
                    pose, rvec, tvec = aruco.estimatePoseCharucoBoard(
                            charucoCorners=charuco_corners, 
                            charucoIds=charuco_ids, 
                            board=CHARUCO_BOARD, 
                            cameraMatrix=cameraMatrix, 
                            distCoeffs=distCoeffs,
                            rvec=rvecs,
                            tvec=tvecs)
                    if pose:
                        # Draw the camera posture calculated from the gridboard
                        QueryImg = aruco.drawAxis(QueryImg, cameraMatrix, distCoeffs, rvec, tvec, 0.3)
            
            # Uncomment If the image size is unknown, set it now
            if not image_size:
                image_size = gray.shape[::-1]
        
            # Uncomment to reproportion the image, maxing width or height at a number (like a 1000)
            proportion = max(QueryImg.shape) / 1000
            QueryImg = cv2.resize(QueryImg, (int(QueryImg.shape[1]/proportion), int(QueryImg.shape[0]/proportion)))

            # Save, Display, Store the written over images
            # cv2.imwrite(os.path.join(directory_pose , 'charuco_pose_frame_' +str(counter)+ '.jpg'), QueryImg)
            img_array.append(QueryImg)
            cv2_imshow(QueryImg)

        # Exit at the end of the video on the 'q' keypress
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
# Get base shape of images to string into a video
height,width,layers=img_array[1].shape

# Create video object to write out to
vid = cv2.VideoWriter(directory_vid_out + 'charuco_pose_output.avi',fourcc, 1, (width,height))

# Write positional images to video
for i in range(len(img_array)):
    vid.write(img_array[i])

# Release everything
cam.release()
vid.release()
cv2.destroyAllWindows()